<a href="https://colab.research.google.com/github/MittalNeha/Extensive_Vision_AI6/blob/main/week1/EVA4_Session_2_neha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function #Forces to use print as a function and not a statement
import torch #pytorch package for python
import torch.nn as nn #import neural network 
import torch.nn.functional as F #convolution functions
import torch.optim as optim #package for the optimizers
from torchvision import datasets, transforms  #package for datasets and computer vision transforms

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -28x28 OUtput - 28x28 RF- 3
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    #28x28   - 28x28    - 5 
#         self.pool1 = nn.MaxPool2d(2, 2)                 #28x28   - 14x14    - 10
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)   #14x14   - 14x14    - 12 
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  #14x14   - 14x14    - 14
#         self.pool2 = nn.MaxPool2d(2, 2)                 #14x14   - 7x7      - 28
#         self.conv5 = nn.Conv2d(256, 512, 3)             #7x7     - 5x5      - 30
#         self.conv6 = nn.Conv2d(512, 1024, 3)            #5x5     - 3x3      - 32
#         self.conv7 = nn.Conv2d(1024, 10, 3)             #3x3     - 1x1      - 34 

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = self.conv7(x)
#         x = x.view(-1, 10)
#         return F.log_softmax(x) #softmax followed by log

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -28x28 OUtput - 28x28 RF- 3
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)    #28x28   - 28x28    - 5 
        self.pool1 = nn.MaxPool2d(2, 2)                 #28x28   - 14x14    - 10
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)   #14x14   - 14x14    - 12 
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)  #14x14   - 14x14    - 14
        self.pool2 = nn.MaxPool2d(2, 2)                 #14x14   - 7x7      - 28
        self.conv5 = nn.Conv2d(64, 128, 3)             #7x7     - 5x5      - 30
        self.conv6 = nn.Conv2d(128, 1024, 3)            #5x5     - 3x3      - 32
        self.conv7 = nn.Conv2d(1024, 10, 3)             #3x3     - 1x1      - 34 

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        x = self.conv7(x)

        x = x.view(-1, 10)
        return F.log_softmax(x) #softmax followed by log


In [4]:
#Initialize the network on CUDA and print the model summary
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 32, 28, 28]           9,248
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
            Conv2d-5           [-1, 64, 14, 14]          36,928
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7            [-1, 128, 5, 5]          73,856
            Conv2d-8           [-1, 1024, 3, 3]       1,180,672
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 1,411,690
Trainable params: 1,411,690
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.74
Params size (MB): 5.39
Estimated Total Size (MB): 6.13
---------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:
torch.manual_seed(1) #seed for generating random numbers
batch_size = 128 
#copy tensors into CUDA pinned memory
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

#Load the MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
from tqdm import tqdm #used for progress bar
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # clear gradients from the last step
        output = model(data)
        loss = F.nll_loss(output, target) #negative log likelihood loss
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

For more experiments after first run

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 30):
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0682, Accuracy: 9783/10000 (97.8300%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0515, Accuracy: 9835/10000 (98.3500%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0370, Accuracy: 9931/10000 (99.3100%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0374, Accuracy: 9931/10000 (99.3100%)



loss=1.1276344594080001e-05 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.29it/s]



Test set: Average loss: 0.0377, Accuracy: 9931/10000 (99.3100%)

